In [21]:
import os
import json
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Layout, Dropdown, BoundedIntText, BoundedFloatText,
    FloatText, Checkbox, Button, Text, Label, Tab, HTML
)
from IPython.display import display


# Basic Setup
1. What are the parameters? What can be default and what can be customized? ✅

## Metadata
Threads & debug output: 
1. ? number of CPU/Processor used for simulation ```num_processors```
2. ? frequncy for debugging ```debug_output_frequency```

Default values:

```num_processors = 4```

```debug_output_frequency = 10```

## PottsCore
All parameters for Cellular Potts Model:
1. Lattice dimensions: ```dim_x, dim_y, dim_z```
2. Number of Monte Carlos Steps: ```steps```
3. Anneal (Number of annealing steps/initial relaxation): ```anneal```
4. Amplitude of pixel copy fluctuations (temp.): ```fluctuation_amplitude```
5. Fluctuation calculation: ```fluctuation_amplitude_function```
6. Boundary conditions: ```boundary_x, boundary_y, boundary_z```
7. Neighborhood order: ```neighbor_order```
8. Random seed: ```random_seed```
9. Lattice geometry: ```lattice_type```
10. Lattice offset (for hexagonal lattices): ```offset ``` 

Default values:

```dim_x, dim_y, dim_z = 1,1,1```

```steps = 0```

```anneal = 0```

```fluctuation_amplitude = 10.0```

```fluctuation_amplitude_function = Min```

```boundary_x, boundary_y, boundary_z = NoFlux```

```neighbor_order = 1```

```random_seed = None```

```lattice_type = Cartesian```

```offset = 0``` 



In [23]:

# Configuration
SAVE_FILE = 'simulation_setup.json'

# Default values
DEFAULTS = {
    "Metadata": {
        "num_processors": 4,
        "debug_output_frequency": 10
    },
    "PottsCore": {
        "dim_x": 100,
        "dim_y": 100,
        "dim_z": 1,
        "steps": 100000,
        "anneal": 0,
        "fluctuation_amplitude": 10.0,
        "fluctuation_amplitude_function": "Min",
        "boundary_x": "NoFlux",
        "boundary_y": "NoFlux",
        "boundary_z": "NoFlux",
        "neighbor_order": 1,
        "random_seed": None,
        "lattice_type": "Cartesian",
        "offset": 0
    },
    "CellType": [
        {"Cell type": "Medium", "freeze": False}
    ]
}

class SimulationSetup:
    def __init__(self):
        self.load_saved_values()
        self.create_widgets()
        self.setup_ui()
        self.setup_event_handlers()
        
    def load_saved_values(self):
        """Load saved values or use defaults"""
        try:
            if os.path.exists(SAVE_FILE):
                with open(SAVE_FILE, 'r') as f:
                    self.saved_values = json.load(f)
            else:
                self.saved_values = DEFAULTS.copy()
        except (json.JSONDecodeError, IOError):
            print("⚠️ JSON file is corrupted or inaccessible. Resetting to defaults.")
            self.saved_values = DEFAULTS.copy()
    
    def create_widgets(self):
        """Create all UI widgets"""
        # Metadata widgets
        self.num_proc = widgets.IntText(
            value=self.saved_values["Metadata"]["num_processors"], min=1,
            description='Number of Processors:',
            style={'description_width': 'initial'}
        )
        
        self.debug_freq = widgets.IntText(
            value=self.saved_values["Metadata"]["debug_output_frequency"], min=1,
            description='Debug Output Frequency:',
            style={'description_width': 'initial'}  
        )
        
        # PottsCore widgets - Updated to match lattice dimensions (0-100 range)
        self.x_slider = widgets.IntSlider(
            value=self.saved_values["PottsCore"]["dim_x"],
            min=0, max=100, description='X:', 
            continuous_update=False,
            style={'description_width': 'initial'}
        )
        
        self.y_slider = widgets.IntSlider(
            value=self.saved_values["PottsCore"]["dim_y"],
            min=0, max=100, description='Y:', 
            continuous_update=False,
            style={'description_width': 'initial'}
        )
        
        self.z_slider = widgets.IntSlider(
            value=self.saved_values["PottsCore"]["dim_z"],
            min=0, max=100, description='Z:', 
            continuous_update=False,
            style={'description_width': 'initial'}
        )
        
        self.steps_input = widgets.IntText(
            value=self.saved_values["PottsCore"]["steps"],
            description='MC Steps:',
            style={'description_width': 'initial'}
        )
        
        self.anneal_input = widgets.FloatText(
            value=self.saved_values["PottsCore"]["anneal"],
            description='Anneal:',
            style={'description_width': 'initial'}
        )
        
        self.fluctuation_slider = widgets.FloatSlider(
            value=self.saved_values["PottsCore"]["fluctuation_amplitude"],
            min=0.0, max=50.0, step=0.1,
            description='Fluctuation:',
            continuous_update=False,
            style={'description_width': 'initial'}
        )
        
        self.flunct_fn_dropdown = widgets.Dropdown(
            options=['Min', 'Max', 'ArithmeticAverage'],
            value=self.saved_values["PottsCore"]["fluctuation_amplitude_function"],
            description='Fluctuation Function:',
            style={'description_width': 'initial'}
        )
        
        self.boundary_x = widgets.Dropdown(
            options=['NoFlux', 'Periodic'],
            value=self.saved_values["PottsCore"]["boundary_x"],
            description='Boundary X:',
            style={'description_width': 'initial'}
        )
        
        self.boundary_y = widgets.Dropdown(
            options=['NoFlux', 'Periodic'],
            value=self.saved_values["PottsCore"]["boundary_y"],
            description='Boundary Y:',
            style={'description_width': 'initial'}
        )
        
        self.boundary_z = widgets.Dropdown(
            options=['NoFlux', 'Periodic'],
            value=self.saved_values["PottsCore"]["boundary_z"],
            description='Boundary Z:',
            style={'description_width': 'initial'}
        )
        
        self.neighbor_order_input = widgets.BoundedIntText(
            value=self.saved_values["PottsCore"]["neighbor_order"],
            min=1, max=20, description='Neighbor Order:',
            style={'description_width': 'initial'}
        )
        
        self.seed_input = widgets.Text(
            value='' if self.saved_values["PottsCore"]["random_seed"] is None 
                 else str(self.saved_values["PottsCore"]["random_seed"]),
            description='Random Seed:',
            placeholder='e.g. 123456',
            style={'description_width': 'initial'}
        )
        
        self.lattice_dropdown = widgets.Dropdown(
            options=['Square', 'Hexagonal'],
            value='Square' if self.saved_values["PottsCore"]["lattice_type"] == 'Cartesian' 
                  else 'Hexagonal',
            description='Lattice Type:',
            style={'description_width': 'initial'}
        )
        
        self.offset_input = widgets.IntText(
            value=self.saved_values["PottsCore"]["offset"],
            description='Offset:',
            style={'description_width': 'initial'}
        )
        
        # CellType widgets
        self.celltype_entries = self.saved_values.get("CellType", DEFAULTS["CellType"].copy())
        self.celltype_display_box = VBox(
            layout=Layout(border='1px solid gray', padding='10px', width='300px')
        )
        # Create HBox for cell type and freeze checkbox
        self.cell_type_row = HBox([
            Dropdown(
                options=["Medium", "Condensing", "NonCondensing", "Customize"],
                value="Medium",
                description="Cell Type:",
                style={'description_width': 'initial'},
                layout=Layout(width='250px')
            )
        ])
        
        self.preset_dropdown = self.cell_type_row.children[0]  # Reference the dropdown
        
        # Add freeze checkbox to the same row
        self.freeze_checkbox = Checkbox(
            value=True,
            description="Freeze",
            indent=False,
            layout=Layout(margin='0 0 0 20px')
        )
        self.cell_type_row.children += (self.freeze_checkbox,)
        
        self.custom_name_input = Text(
            description="Name:",
            placeholder="e.g. T1",
            style={'description_width': 'initial'},
            layout=Layout(width='250px')
        )
        self.custom_name_input.layout.display = 'none'
        
        self.add_button = Button(
            description="Add",
            button_style="success",
            layout=Layout(width='80px', margin='10px 0 0 0')
        )
        
        # Create reset buttons for each tab
        self.reset_metadata_button = Button(
            description="Reset Tab",
            button_style='warning',
            layout=Layout(width='100px')
        )
        
        self.reset_potts_button = Button(
            description="Reset Tab",
            button_style='warning',
            layout=Layout(width='100px')
        )
        
        self.reset_ct_button = Button(
            description="Reset Tab",
            button_style='warning',
            layout=Layout(width='100px')
        )
        
        self.reset_button = Button(
            description="Reset All to Defaults",
            button_style='danger'
        )
    
    def setup_ui(self):
        """Setup the UI layout"""
        # Metadata tab with reset button
        self.metadata_box = VBox(
            [self.num_proc, self.debug_freq, self.reset_metadata_button],
            layout=Layout(padding='10px')
        )
        
        # PottsCore tab with reset button
        self.potts_box = VBox([
            HTML("<b>Dimensions:</b>"),
            HBox([self.x_slider, self.y_slider, self.z_slider]),
            HTML("<b>Core Parameters:</b>"),
            self.steps_input, self.anneal_input, 
            self.fluctuation_slider, self.flunct_fn_dropdown,
            HTML("<b>Boundaries:</b>"),
            HBox([self.boundary_x, self.boundary_y, self.boundary_z]),
            HTML("<b>Advanced:</b>"),
            self.neighbor_order_input, self.lattice_dropdown, 
            self.offset_input, self.seed_input,
            self.reset_potts_button
        ], layout=Layout(padding='10px'))
        
        # CellType tab with reset button
        self.celltype_box = VBox([
            HBox([
                VBox([
                    Label("Current Cell Types:", style={'font_weight': 'bold'}),
                    self.celltype_display_box,
                    self.reset_ct_button
                ], layout=Layout(width='320px', padding='0 20px 0 0')),
                VBox([
                    Label("Add Cell Type:", style={'font_weight': 'bold'}),
                    self.cell_type_row,  # Use the HBox with dropdown and checkbox
                    self.custom_name_input,
                    self.add_button
                ])
            ])
        ], layout=Layout(padding='10px'))
        
        # Main tabs
        self.tabs = Tab()
        self.tabs.children = [self.metadata_box, self.potts_box, self.celltype_box]
        self.tabs.set_title(0, 'Basic Setup')
        self.tabs.set_title(1, 'Potts Core')
        self.tabs.set_title(2, 'Cell Types')
        #self.tabs.set_title(3, 'Volume')
        #self.tabs.set_title(4, 'Chemical Field')
        #self.tabs.set_title(5, 'Cell Properties & Behaviors')
        #self.tabs.set_title(6, 'Initilization')
        #self.tabs.set_title(7, 'Simulation')
        
        # Update initial display
        self.update_celltype_display()
    
    def setup_event_handlers(self):
        """Setup all event handlers"""
        # Save triggers
        for w in [
            self.num_proc, self.debug_freq, 
            self.x_slider, self.y_slider, self.z_slider,
            self.steps_input, self.anneal_input,
            self.fluctuation_slider, self.flunct_fn_dropdown,
            self.neighbor_order_input, self.boundary_x,
            self.boundary_y, self.boundary_z,
            self.lattice_dropdown, self.offset_input,
            self.seed_input
        ]:
            w.observe(lambda _: self.save_to_json(), names='value')
        
        # CellType management
        self.preset_dropdown.observe(self.toggle_custom_input, names='value')
        self.add_button.on_click(self.on_add_clicked)
        self.reset_ct_button.on_click(lambda _: self.reset_tab('CellType'))
        self.reset_metadata_button.on_click(lambda _: self.reset_tab('Metadata'))
        self.reset_potts_button.on_click(lambda _: self.reset_tab('PottsCore'))
        self.reset_button.on_click(self.reset_to_defaults)
    
    def current_config(self):
        """Get current configuration from widgets"""
        return {
            "Metadata": {
                "num_processors": self.num_proc.value,
                "debug_output_frequency": self.debug_freq.value
            },
            "PottsCore": {
                "dim_x": self.x_slider.value,
                "dim_y": self.y_slider.value,
                "dim_z": self.z_slider.value,
                "steps": self.steps_input.value,
                "anneal": self.anneal_input.value,
                "fluctuation_amplitude": self.fluctuation_slider.value,
                "fluctuation_amplitude_function": self.flunct_fn_dropdown.value,
                "boundary_x": self.boundary_x.value,
                "boundary_y": self.boundary_y.value,
                "boundary_z": self.boundary_z.value,
                "neighbor_order": self.neighbor_order_input.value,
                "random_seed": int(self.seed_input.value) if self.seed_input.value.strip() else None,
                "lattice_type": 'Cartesian' if self.lattice_dropdown.value == 'Square' else 'Hexagonal',
                "offset": self.offset_input.value
            },
            "CellType": self.celltype_entries
        }
    
    def save_to_json(self, *_):
        """Save current configuration to JSON"""
        with open(SAVE_FILE, 'w') as f:
            json.dump(self.current_config(), f, indent=4)
    
    def update_celltype_display(self):
        """Update the cell type display box"""
        items = []
        for i, entry in enumerate(self.celltype_entries):
            label_str = entry["Cell type"]
            if entry["freeze"]:
                label_str += " (frozen)"
            label = Label(label_str, layout=Layout(flex='1 1 auto'))
            remove_btn = Button(
                description="Remove", 
                button_style='danger', 
                layout=Layout(width='80px')
            )
            
            def make_remove_handler(index):
                def handler(_):
                    del self.celltype_entries[index]
                    self.update_celltype_display()
                    self.save_to_json()
                return handler
            
            remove_btn.on_click(make_remove_handler(i))
            items.append(HBox(
                [label, remove_btn], 
                layout=Layout(justify_content='space-between')
            ))
        
        self.celltype_display_box.children = items
        self.save_to_json()
    
    def toggle_custom_input(self, change):
        """Toggle custom name input visibility"""
        self.custom_name_input.layout.display = 'block' if change['new'] == "Customize" else 'none'
    
    def on_add_clicked(self, _):
        """Handle add cell type button click"""
        selected = self.preset_dropdown.value
        name = (self.custom_name_input.value.strip() 
                if selected == "Customize" else selected)
        
        if not name:
            self.custom_name_input.placeholder = "Please enter a name!"
            return
        
        self.celltype_entries.append({
            "Cell type": name, 
            "freeze": self.freeze_checkbox.value
        })
        
        self.update_celltype_display()
        self.custom_name_input.value = ""
        self.freeze_checkbox.value = True
    
    def reset_tab(self, tab_name):
        """Reset a specific tab to defaults"""
        if tab_name == "Metadata":
            self.num_proc.value = DEFAULTS["Metadata"]["num_processors"]
            self.debug_freq.value = DEFAULTS["Metadata"]["debug_output_frequency"]
        
        elif tab_name == "PottsCore":
            self.x_slider.value = DEFAULTS["PottsCore"]["dim_x"]
            self.y_slider.value = DEFAULTS["PottsCore"]["dim_y"]
            self.z_slider.value = DEFAULTS["PottsCore"]["dim_z"]
            self.steps_input.value = DEFAULTS["PottsCore"]["steps"]
            self.anneal_input.value = DEFAULTS["PottsCore"]["anneal"]
            self.fluctuation_slider.value = DEFAULTS["PottsCore"]["fluctuation_amplitude"]
            self.flunct_fn_dropdown.value = DEFAULTS["PottsCore"]["fluctuation_amplitude_function"]
            self.boundary_x.value = DEFAULTS["PottsCore"]["boundary_x"]
            self.boundary_y.value = DEFAULTS["PottsCore"]["boundary_y"]
            self.boundary_z.value = DEFAULTS["PottsCore"]["boundary_z"]
            self.neighbor_order_input.value = DEFAULTS["PottsCore"]["neighbor_order"]
            self.seed_input.value = ''
            self.lattice_dropdown.value = 'Square' if DEFAULTS["PottsCore"]["lattice_type"] == 'Cartesian' else 'Hexagonal'
            self.offset_input.value = DEFAULTS["PottsCore"]["offset"]
        
        elif tab_name == "CellType":
            self.celltype_entries = DEFAULTS["CellType"].copy()
            self.update_celltype_display()
    
    def reset_to_defaults(self, _):
        """Reset all settings to defaults"""
        self.reset_tab("Metadata")
        self.reset_tab("PottsCore")
        self.reset_tab("CellType")
    
    def show(self):
        """Display the UI"""
        display(VBox([self.tabs, self.reset_button]))

# Main execution
if __name__ == "__main__":
    try:
        setup = SimulationSetup()
        setup.show()
    except Exception as e:
        print(f"Error creating interface: {e}")